<a href="https://colab.research.google.com/github/doraemonidol/bert-crf/blob/master/punktuation_ner_2k_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# import sys
# sys.path.append('/content/gdrive/MyDrive/Colab/punktuation-ner')

In [3]:
!pip install forgebox==0.4.18.5 pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 45.1 MB/s eta 0:00:00


# Punctuation NER

In [4]:
# Forgebox Imports
from forgebox.imports import *
from forgebox.category import Category
import pytorch_lightning as pl
from transformers import AutoTokenizer, BertForTokenClassification, BertModel
from transformers import pipeline
from typing import List
import re
from torch.utils.data import DataLoader, Dataset

In [5]:
DATA = r'/content/gdrive/MyDrive/Colab/punktuation-ner/data'

In [6]:
# In data/couplets/Chinese_couplet_dataset_sample_2K.tsv, the format is as follows:
# first,second,label
# 明 有 通 人 著 赤 雅                                                                            ,汉 称 孝 子 褒 黄 香,0000000
# 栖 霞 山 上 栖 霞 寺                                                                            ,建 业 城 中 建 业 人,0000000
# 修 身 如 执 玉                                                                                  ,行 善 胜 遗 金,00000
# 闲 云 归 岫 连 峰 暗                                                                            ,飞 瀑 垂 空 漱 石 凉,0000000
# 雪 寂 春 薄 ， 谁 怜 取 寒 香 一 抹                                                             ,夜 深 人 静 ， 伊 独 饮 醉 苦 千 般,000010000000

# Load the data and use train_test_split to split the data into training and validation sets, save the data to the data folder
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(f"{DATA}/Chinese_couplet_dataset_sample_2K.tsv", encoding="utf-8", sep=",")

# Remove the space in the first, second
df["first"] = df["first"].apply(lambda x: x.replace(" ", ""))
df["second"] = df["second"].apply(lambda x: x.replace(" ", ""))

print("Length before: " + str(len(df)))
df = df.groupby('label').filter(lambda x: len(x) > 1)
print("Length after: " + str(len(df)))

# # Combine the first and second columns into a new column text
df["text"] = df["first"] + '\n' + df["second"]

# # Remove the first, second columns
df = df.drop(columns=["first", "second"])

df_train, df_val = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])

with open(f"{DATA}/split/train.txt", "w") as f:
    for text in df_train["text"]:
        f.write(text + "\n")

with open(f"{DATA}/split/valid.txt", "w") as f:
    for text in df_val["text"]:
        f.write(text + "\n")

Length before: 2000
Length after: 1909


In [ ]:
# In data/train-fixed.txt, the format is as follows:
# input	output
# 年年月月動動儉儉積積撒撒家家富富裕裕	年年月月,動動,儉儉,積積,撒撒,家家,富富,裕裕
# 去去來來取取存存角角元元户户歡歡欣欣	去去來來,取取,存存,角角,元元,户户,歡歡,欣欣

# Only get the output columns, give each row a label by a define functions. Load the data and use train_test_split to split the data into training and validation sets, save the data to the data folder.

def label(x):
    punctuations = ["，", "。", "！", "？", "、", "：", "；", "「", "」", "『", "』", "（", "）", "《", "》", "〈", "〉", "【", "】", "〔", "〕", "［", "］", "｛", "｝", "﹙", "﹚", "﹛", "﹜", "﹝", "﹞", "‘", "’", "“", "”", "〝", "〞", "‵", "′", "〃", "‶", "‷", "﹁", "﹂", "﹃", "﹄", "《", "》", "〈", "〉", "「", "」", "『", "』", "〖", "〗", "【", "】", "±", "×", "÷", "∶", "∣", "∥", "∧", "∨", "∩", "∪", "∫", "∮", "∝", "∞", "∠", "∟", "∥", "∧", "∨", "∩", "∪", "∫", "∮", "∝", "∞", "∠", "∟", "∥", "∧", "∨", "∩", "∪", "∫", "∮", "∝", "∞", "∠", "∟", "∥", "∧", "∨", "∩", "∪", "∫", "∮", "∝", "∞", "∠", "∟", "∥", "∧", "∨", "∩", "∪", "∫", "∮", "∝", "∞", "∠", "∟", "∥", "∧", "∨", "∩", "∪", "∫", "∮", "∝", "∞", "∠", "∟", "∥", "∧",]
    label = []
    for punctuation in punctuations:
        if punctuation in x:
            label.append("1")
        else:
            label.append("0")

df = pd.read_csv(f"{DATA}/train-fixed.txt", encoding="utf-8", sep="\t")
df.drop(columns=["input"], inplace=True)
df["label"] = df["output"].apply(label)

df_train, df_val = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

with open(f"{DATA}/split/train.txt", "w") as f:
    for text in df_train["output"]:
        f.write(text + "\n")

with open(f"{DATA}/split/valid.txt", "w") as f:
    for text in df_val["output"]:
        f.write(text + "\n")

## Read Metadata

In [ ]:
# META = pd.read_csv(DATA/"meta.csv")

In [7]:
LABELS = ['train.txt', 'valid.txt']

In [8]:
punkt_regex = r'[^\w\s]'

def position_of_all_punctuation(x):
    return [m.start() for m in re.finditer(punkt_regex, x)]

# simplify the punctuation
eng_punkt_to_cn_dict = {
    ".": "1",
    ",": "1",
    ":": "1",
    ";": "1",
    "?": "1",
    "!": "1",
    "“": "1",
    "”": "1",
    "‘": "1",
    "’": "1",
    "「": "1",
    "」": "1",
    "『": "1",
    "』": "1",
    "（": "1",
    "）": "1",
    "《": "1",
    "》": "1",
    "［": "1",
    "］": "1",
    }

def translate_eng_punkt_to_cn(char):
    if char == "0":
        return char
    if char in eng_punkt_to_cn_dict.values():
        return char
    result = eng_punkt_to_cn_dict.get(char)
    if result is None:
        return "1"
    return result

def punct_ner_pair(sentence):
    positions = position_of_all_punctuation(sentence)
    x = re.sub(punkt_regex, '', sentence)
    y = list("0"*len(x))

    for i, p in enumerate(positions):
        y[p-i-1] = sentence[p]
    p_df = pd.DataFrame({"x":list(x), "y":y})
    p_df["y"] = p_df["y"].apply(translate_eng_punkt_to_cn)
    return p_df

In [11]:
ALL_LABELS = ["0", "1"] #+list(eng_punkt_to_cn_dict.values())

In [10]:
print(ALL_LABELS)

['0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']


In [12]:
cates = Category(ALL_LABELS)

In [13]:
class PunctDataset(Dataset):
    def __init__(
        self,
        data_dir: Path,
        filelist: List[str],
        num_threads: int = 8,
        length: int = 1000,
        size: int = 540
    ):
        """
        Args:
            - filelist: list of file names
            - The dataset will open ```num_threads``` files, and hold
                in memory simoultaneously.
            - num_threads: number of threads to read files,
            - length: number of sentences per batch
            - size: number of characters per sentence
        """
        self.data_dir = Path(data_dir)
        self.filelist = filelist
        self.num_threads = num_threads
        self.length = length
        # open file strings, index is mod of num_threads
        self.current_files = dict(enumerate([""]*length))
        self.string_index = dict(enumerate([0]*length))
        self.to_open_idx = 0
        self.size = size
        self.get_counter = 0
        self.return_string = False

    def __len__(self):
        return self.length

    def __repr__(self):
        return f"PunctDataset: {len(self)}, on {len(self.filelist)} files"

    def new_file(self, idx_mod):
        filename = self.filelist[self.to_open_idx]
        with open(self.data_dir/filename, "r", encoding="utf-8") as f:
            self.current_files[idx_mod] = f.read()

        self.to_open_idx += 1

        # reset to open article file index
        if self.to_open_idx >= len(self.filelist):
            self.to_open_idx = 0

        # reset string_index within new article file
        self.string_index[idx_mod] = 0

        # if self.to_open_idx % 500 == 0:
        #     print(f"went through files:\t{self.to_open_idx}")

    def __getitem__(self, idx):
        idx_mod = self.get_counter % self. num_threads

        if self.string_index[idx_mod] >= len(self.current_files[idx_mod]):
            self.new_file(idx_mod)
        string_idx = self.string_index[idx_mod]

        # slicing a sentence
        sentence = self.current_files[idx_mod][string_idx:string_idx+self.size]

        # move the string_index within current article file
        self.string_index[idx_mod] += self.size

        # move the get_counter
        self.get_counter += 1
        p_df = punct_ner_pair(sentence)
        return list(p_df.x), list(p_df.y)

    def align_offsets(
        self,
        inputs,
        text_labels: List[List[str]],
        words: List[List[str]]
    ):
        """
        inputs: output if tokenizer
        text_labels: labels in form of list of list of strings
        words: words in form of list of list of strings
        """
        labels = torch.zeros_like(inputs.input_ids).long()
        labels -= 100
        text_lables_array = np.empty(labels.shape, dtype=object)
        words_array = np.empty(labels.shape, dtype=object)
        max_len = inputs.input_ids.shape[1]

        # print("Input_ids: ", inputs.input_ids)

        # print("Max Len: ", max_len)

        # print("Text Labels: ", text_labels)

        for row_id, input_ids in enumerate(inputs.input_ids):
            word_pos = inputs.word_ids(row_id)
            # print("Word Pos: ", word_pos)
            for idx, pos in enumerate(word_pos):
                # print("index: ", idx)
                if pos is None:
                    # print("Pos is None")
                    continue
                labels[row_id, idx] = self.cates.c2i[text_labels[row_id][pos]]
                if self.return_string:
                    text_lables_array[row_id,
                                        idx] = text_labels[row_id][pos]
                    words_array[row_id, idx] = words[row_id][pos]

        inputs['labels'] = labels

        if self.return_string:
            inputs['text_labels'] = text_lables_array.tolist()
            inputs['word'] = words_array.tolist()

        # for input_id in inputs['input_ids']:
        #     print("InPuT_iD: ", input_id)
        #     print("Word: ", self.tokenizer.convert_ids_to_tokens(input_id))
        #     print("Word Pos: ", inputs.word_ids(0))
        # for label in inputs['labels']:
        #     print("Label: ", label)

        return inputs

    def collate_fn(self, data):
        """
        data: list of tuple
        """
        words, text_labels = zip(*data)

        inputs = self.tokenizer(
            list(words),
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=self.max_len,
            is_split_into_words=True,
            return_offsets_mapping=True,
            add_special_tokens=False,
        )

        # print("Original words", words)

        return self.align_offsets(inputs, text_labels, words)

    def dataloaders(self, tokenizer, cates, max_len: int = 512, batch_size: int = 32):
        self.tokenizer = tokenizer
        self.cates = cates
        self.max_len = max_len
        return DataLoader(
            self,
            batch_size=batch_size,
            shuffle=False,
            collate_fn=self.collate_fn,
        )

    def split(self, ratio: float = 0.9, n_splits: int = 5):
      """
      Split the dataset into train and valid using StratifiedKFold
      """

      train_filelist = ["train.txt"]
      valid_filelist = ["valid.txt"]

      train_dataset = PunctDataset(
          self.data_dir,
          train_filelist,
          num_threads=self.num_threads,
          length=int(self.length * ratio),
          size=self.size,
      )
      valid_dataset = PunctDataset(
          self.data_dir,
          valid_filelist,
          num_threads=self.num_threads,
          length=int(self.length * (1 - ratio)),
          size=self.size,
      )
      return train_dataset, valid_dataset

Create dataset object

* Length is the length of the epoch
* Size: is the sequence length
* num_threads: num of files that is opening at the same time

In [14]:
ds = PunctDataset(DATA + '/split', LABELS, num_threads=1, length=4000, size=512)
train_ds, valid_ds = ds.split(0.8)

### lightning data module

In [15]:
class PunctDataModule(pl.LightningDataModule):
    def __init__(self, train_ds, valid_ds, tokenizer, cates,
    max_len=512, batch_size=32):
        super().__init__()
        self.train_ds, self.valid_ds = train_ds, valid_ds
        self.tokenizer = tokenizer
        self.cates = cates
        self.max_len = max_len
        self.batch_size = batch_size

    def split_data(self):

        return train_ds, valid_ds

    def train_dataloader(self):
        return self.train_ds.dataloaders(
            self.tokenizer,
            self.cates,
            self.max_len,
            self.batch_size,
        )

    def val_dataloader(self):
        return self.valid_ds.dataloaders(
            self.tokenizer,
            self.cates,
            self.max_len,
            self.batch_size*4)

## Load Pretrained

In [16]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
from forgebox.thunder.callbacks import DataFrameMetricsCallback
from forgebox.hf.train import NERModule

In [18]:


# Define BERT-CRF Model using PyTorch Lightning
class BERT_CRF(pl.LightningModule):
    def __init__(self, model, num_labels, learning_rate=1e-5):
        super().__init__()
        self.save_hyperparameters(ignore=['model'])

        self.model = model
        self.crf = CRF(num_labels, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.model.config.hidden_size, num_labels)
        self.lr = learning_rate

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        sequence_output = output.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        logits = self.fc(sequence_output)
        if labels is not None:
            loss = -self.crf(logits, labels, mask=attention_mask.bool(), reduction='mean')
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, logits = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss)
        self.log('train_acc', self.crf.accuracy(logits, labels, attention_mask.byte()))
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, logits = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss)
        self.log('val_acc', self.crf.accuracy(logits, labels, attention_mask.byte()))
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        return optimizer


Load pretrained model with proper num of categories

In [19]:
model = BertForTokenClassification.from_pretrained("bert-base-chinese", num_labels=len(cates),)

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
print(train_ds.length)
print(valid_ds.length)

3200
799


In [21]:
data_module = PunctDataModule(train_ds, valid_ds, tokenizer, cates,
                              batch_size=32,)

### Run data pipeline

In [22]:
inputs = next(iter(data_module.val_dataloader()))
print(inputs.input_ids)

tensor([[3189, 4212, 7198,  ...,    0,    0,    0],
        [6843, 4242, 1898,  ...,    0,    0,    0],
        [7412, 5782, 2255,  ..., 7491,    0,    0],
        ...,
        [3717,  756, 5682,  ..., 7414,    0,    0],
        [6816,  881,  782,  ...,    0,    0,    0],
        [4080, 3958, 3726,  ...,    0,    0,    0]])


In [23]:
inputs.input_ids.shape

torch.Size([128, 439])

In [24]:
inputs.labels.shape

torch.Size([128, 439])

## NER tranining module

In [25]:
module = NERModule(model)

In [26]:
save_callback = pl.callbacks.ModelCheckpoint(
    dirpath=f"content/ckpoint",
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
)
df_show = DataFrameMetricsCallback()

Reset the configure_optimizers function

In [27]:
def configure_optimizers(self):
        # discriminative learning rate
    param_groups = [
            {'params': self.model.bert.parameters(), 'lr': 5e-6},
            {'params': self.model.classifier.parameters(), 'lr': 1e-3},
        ]
    optimizer = torch.optim.Adam(param_groups, lr=1e-3)
    return optimizer

NERModule.configure_optimizers = configure_optimizers

In [28]:
import torch
print(torch.cuda.is_available())

True


Trainer

In [29]:
trainer = pl.Trainer(
    accelerator='gpu',
    devices=1,
    max_epochs=30,
    callbacks=[df_show, save_callback],
    )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [30]:
trainer.fit(module, datamodule=data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | BertForTokenClassification | 101 M  | eval
------------------------------------------------------------
101 M     Trainable params
0         Non-trainable params
101 M     Total params
406.714   Total estimated model params size (MB)
0         Modules in train mode
228       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 400: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 500: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

## Load the best model

In [ ]:
module = NERModule.load_from_checkpoint(save_callback.best_model_path, model=model)

In [ ]:
module.model.config.id2label = dict(enumerate(cates.i2c))
module.model.config.label2id = cates.c2i.dict

In [ ]:
from transformers import pipeline

In [ ]:
module.model = module.model.eval()
module.model = module.model.cpu()

In [ ]:
# prompt: Store the model

torch.save(module.model, '/content/gdrive/MyDrive/Colab/punktuation-ner/best_ckpoint/punct_model_2.pth')


In [ ]:
ner = pipeline("ner",module.model,tokenizer=tokenizer)

In [ ]:
def mark_sentence(x: str):
    outputs = ner(x)
    print(outputs)
    print("hello")
    x_list = list(x)
    for i, output in enumerate(outputs):
        x_list.insert(output['end']+i, output['entity'])
    return "".join(x_list)

In [ ]:
mark_sentence("洛水神龜單應兆天數九地數九九九八十一數數數混成三大道道合元始天尊一成有感")

In [ ]:
mark_sentence("""郡邑置夫子庙于学以嵗时释奠盖自唐贞观以来未之或改我宋有天下因其制而损益之姑苏当浙右要区规模尤大更建炎戎马荡然无遗虽修学宫于荆榛瓦砾之余独殿宇未遑议也每春秋展礼于斋庐已则置不问殆为阙典今寳文阁直学士括苍梁公来牧之明年实绍兴十有一禩也二月上丁修祀既毕乃愓然自咎揖诸生而告之曰天子不以汝嘉为不肖俾再守兹土顾治民事神皆守之职惟是夫子之祀教化所基尤宜严且谨而拜跪荐祭之地卑陋乃尔其何以掲防妥灵汝嘉不敢避其责曩常去此弥年若有所负尚安得以罢輭自恕复累后人乎他日或克就绪愿与诸君落之于是谋之僚吏搜故府得遗材千枚取赢资以给其费鸠工庀役各举其任嵗月讫工民不与知像设礼器百用具修至于堂室廊序门牖垣墙皆一新之""")

In [ ]:
# import pandas as pd
# df = pd.read_csv('test.txt', sep='\t')

# text_list = df['input'].tolist()[0:]
# predicted = []
# for text in text_list:
#     predicted.append(mark_sentence(text))

# # predicted save to csv
# df2 = pd.DataFrame(predicted)
# # output to csv but ignore header
# df2.to_csv('predicted.txt', index=False, header=False)
# df2 = pd.read_csv('predicted.txt', sep='\t', header=None)
# df2.columns = ['predicted']

# punctuation = ['，', '。', '！', '？', '；', '：', '、', '「', '」', '『', '』', '（', '）', '〔', '〕', '【', '】', '《', '》', '〈', '〉', '﹏', '＿', '～', '—', '…', '‥', '﹑', '﹔', '﹖', '﹪', '﹙', '﹚', '﹛', '﹜', '﹟', '﹠', '﹡', '﹢', '﹣', '﹤', '﹥', '﹦', '﹨', '﹩', '﹪', '﹫', '＃', '＄', '％', '＆', '＊', '＋', '－', '／', '＜', '＝', '＞', '＠', '＾', '＿', '｀', '｜', '～', '∕', '∥']

# predicted_parsed = []
# for text in df2['predicted']:
#     i = 0
#     line = []
#     while i < len(text) - 1:
#         if text[i + 1] in punctuation:
#             line.append(1)
#             i += 1
#         else:
#             line.append(0)
#         i += 1
#     if line[-1] != 1:
#         line.append(1)
#     predicted_parsed.append(' - '.join([str(x) for x in line]))

# df3 = pd.DataFrame(predicted_parsed)
# df3.to_csv('predicted_parsed.txt', index=False, header=False)